In [62]:
from pwn import *
from tqdm import tqdm

r = remote('prob16.geekgame.pku.edu.cn', 10016)
token = b'xxx'
r.sendline(token)

nums = []

l = r.recvline()
l = l.split(b" ")[-1]
nums.append(int(l[:-1]))


for _ in tqdm(range(3000)):
    r.sendline(b'')
    l = r.recvline()
    nums.append(int(l[:-1]))


[x] Opening connection to prob16.geekgame.pku.edu.cn on port 10016
[x] Opening connection to prob16.geekgame.pku.edu.cn on port 10016: Trying 115.27.246.20
[+] Opening connection to prob16.geekgame.pku.edu.cn on port 10016: Done


100%|██████████| 3000/3000 [00:01<00:00, 1706.77it/s]


In [89]:
min(nums), max(nums)

(173459, 4289372821)

In [63]:
def temper(tmp):
    tmp ^= (tmp >> 11)
    tmp ^= (tmp << 7) & 0x9d2c5680
    tmp ^= (tmp << 15) & 0xefc60000
    tmp ^= (tmp >> 18)
    return tmp

In [90]:
def twister(n0, n1, n397):
    y = (n0 & 0x80000000) + (n1 & 0x7fffffff)
    ns = n397 ^ (y >> 1)
    if y % 2:
        ns ^= 0x9908b0df
    return ns

In [66]:
class TemperInverser:
    __b = 0x9d2c5680
    __c = 0xefc60000
    __kMaxBits = 0xffffffff

    def __inverse_right_shift_xor(self, value, shift):
        i, result = 0, 0
        while i * shift < 32:
            part_mask = ((self.__kMaxBits << (32 - shift)) & self.__kMaxBits) >> (i * shift)
            part = value & part_mask
            value ^= part >> shift
            result |= part
            i += 1
        return result

    def __inverse_left_shift_xor(self, value, shift, mask):
        i, result = 0, 0
        while i * shift < 32:
            part_mask = (self.__kMaxBits >> (32 - shift)) << (i * shift)
            part = value & part_mask
            value ^= (part << shift) & mask
            result |= part
            i += 1
        return result

    def __inverse_temper(self, tempered):
        value = tempered
        value = self.__inverse_right_shift_xor(value, 18)
        value = self.__inverse_left_shift_xor(value, 15, self.__c)
        value = self.__inverse_left_shift_xor(value, 7, self.__b)
        value = self.__inverse_right_shift_xor(value, 11)
        return value

    def __call__(self, tempered):
        return self.__inverse_temper(tempered)

inverser = TemperInverser()

In [121]:
# 生成包含大小写字母和数字的集合
char_set = string.ascii_letters + string.digits
char_set += " _{}"
char_num = [ord(x) for x in char_set]
print(char_set, len(char_set))

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 _{} 66


In [141]:
state = [0] * 624
s = "flag{MT19937_cAN_BE_ATTaCKED}"
for i in range(len(s)):
    state[i] = inverser(nums[i] - ord(s[i]))
    
for i in range(len(s)-1, 100):
    state[i] = inverser(nums[i] - ord(s[i]))
    for z in char_num:
        s5 = inverser(nums[i+1] - z)
        able = False
        for x in char_num:
            for y in char_num:
                nsi = inverser(nums[i+624] - x)
                s397 = inverser(nums[i+397] - y)
                if twister(state[i], s5, s397) == nsi:
                    able = True
                    #print(chr(x), chr(y))
                    break
        if able:
            s += chr(z)
            print(i, chr(z))
            break

22 a
23 C
24 K
25 E
26 D
27 }


IndexError: string index out of range